In [ ]:
import sqlite3
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
import requests
from pprint import pprint
import wget
import pandas as pd
import json
import zipfile
import os
import ffmpeg

In [ ]:
! pip install ffmpeg

In [ ]:
session = requests.session()

In [ ]:
ua = UserAgent(verify_ssl=False)

In [ ]:
corpora = ['don_rnd', 'opochka', 'keba']
links = []
parsed_pages = {}
for corpus in corpora:
    helper = []
    url = f'http://lingconlab.ru/{corpus}/files_html/'
    req = session.get(url, headers={'User-Agent': ua.random})
    soup = BeautifulSoup(req.text, "html.parser")
    for i in soup.findAll('a', attrs={'href': re.compile(".html$")}):
        helper.append(url+i.get('href'))
    links.extend(helper[:3])
    parsed_pages[corpus] = helper[:3]

In [ ]:
!mkdir audio_files

In [ ]:
def get_audio_and_meta(corpus, parsed_links, info):
    for informant in parsed_links:      
        req2 = session.get(informant, headers={'User-Agent': ua.random})
        req2.encoding = 'utf8'
        soup2 = BeautifulSoup(req2.text, "html.parser")
        flag = ''
        for i in soup2.findAll('a', attrs={'href': re.compile("javascript:play")}):
            if corpus == 'opochka':
                text = i.previous_sibling.previous_sibling.previous_sibling
            else:
                text = i.previous_sibling
            get_b = i.find_previous_sibling('b').text
            id_full =  i.get('href').strip("javascript:play_sound('").strip("');")
            download_link = f'http://lingconlab.ru/{corpus}/OUT/' + id_full + '.wav'
            filename = 'audio_files/' + id_full + '.wav'
            #text = i.previous_sibling
            id_inf = id_full.split('-')
            inf, start, end = id_inf[0], id_inf[1], id_inf[2]
            if get_b:
                flag = str(get_b).replace(':', '')
            info[id_full] = {'informant': flag, 'start': int(start) / 1000, 'end': int(end) / 1000, 
                             'filename': filename, 'corpus': corpus, 'text': text, 'gender': '', 'age': ''}

            try:
                resp = wget.download(download_link, filename)
                print('written')
            except:
                print(download_link)
    return info

In [ ]:
info = {}
for corpus in parsed_pages:
    get_audio_and_meta(corpus, parsed_pages[corpus], info)

Донские - 6200
Опочка - 2600
Кеба - 1500

In [ ]:
df = pd.DataFrame.from_dict(info, orient='index')

In [ ]:
df['informant'] = df['informant'].str.strip(' ')

In [ ]:
def get_corpus(informant, corpus):
    if 'Interviewer' in informant:
        corpus = 'standart'
    return corpus

In [ ]:
df['corpus'] = df.apply(lambda x: get_corpus(x['informant'], x['corpus']), axis=1)

In [ ]:
df

In [ ]:
df.to_csv('data.csv')

## Опочка
1. Фонетика: яканье, у --> ъ в заударном слоге, дзеканье (т', д' --> тс', дз')
2. Морфология: повелительное наклонение -- бежи, бежите, личное метоимение 3л ед.ч --> йон, йону
## Донские
1. Фонетика: аканье, "доришь, котишь, плотишь"
2. Морфология: средний род --> муж или жен

## Стандартный русский

In [ ]:
page = 'http://spokencorpora.ru/showcorpus.py?dir=01life'

In [ ]:
page_parsed = session.get(page, headers={'User-Agent': ua.random})
page_parsed.encoding = 'utf8'
soup3 = BeautifulSoup(page_parsed.text, "html.parser")

In [ ]:
get_metas = soup3.findAll('script', attrs={'type': 'text/javascript'})[-1].text

In [ ]:
!mkdir standart

In [ ]:
meta_data = json.loads(re.search("datastr: \\'(.+?)\\'", get_metas).group(1))

In [ ]:
normal_way = {}
for i in meta_data['rows']:
    name = i['cell'][0]
    if i['cell'][1] == 'Ж':
        gender = 'f'
    else:
        gender = 'm'
    download_link = f'http://spokencorpora.ru/showtrans.py?file=01life/{name}'
    normal_way[i['cell'][0]] = {'informant': i['cell'][0], 'start': 0, 'end': 0, 'filename': '', 
                                'corpus': 'standart', 'text': '', 'gender': gender, 'age': i['cell'][2] }

In [ ]:
for i in os.listdir('./standart'):
    if i.endswith('.zip'):
        with zipfile.ZipFile(f'./standart/{i}', 'r') as zip_ref:
            zip_ref.extractall('./standart')

In [ ]:
! rm ./standart/*.zip

In [ ]:
!pip install speach

In [ ]:
from speach import elan

In [ ]:
eaf = elan.read_eaf('./standart/Sib_01-f.eaf')

In [ ]:
!mkdir trimmed

In [ ]:
again = {}
regex = re.compile('[^a-zA-Z]')
for i in normal_way:
    eaf = elan.read_eaf(f'./standart/{i}.eaf')
    gender = normal_way[i]['gender']
    age = normal_way[i]['age']
    audiofile_path = f'./standart/{i}.wav'
    for j in eaf['Text']:
        start, end, text = j.from_ts.value / 1000, j.to_ts.value / 1000, j.text
        initial = f'{i}_{start}_{end}'
        filename = f'./trimmed/{initial}.wav'
        again[initial] = {'informant': i, 'start': start, 'end': end, 'filename': filename, 
                                'corpus': 'standart', 'text': text, 'gender': gender, 'age': age}
        
        !ffmpeg -i {audiofile_path} -ss {str(start)} -to {str(end)} -ar 16000 {filename}

In [ ]:
df2 = pd.DataFrame.from_dict(again, orient='index')
df2.to_csv('standart_data.csv')

In [ ]:
! zip -r audio_files.zip audio_files

In [ ]:
! zip -r trimmed.zip trimmed

In [ ]:
!zip all_data.zip audio_files.zip trimmed.zip standart_data.csv data.csv